In [1]:
import pandas as pd

In [2]:
code_redi_df = pd.read_csv("dataset/Code_Red_I.csv")
slammer_df = pd.read_csv("dataset/Slammer - Named Features.csv")
nimda_df = pd.read_csv("dataset/Nimda.csv")

#### Combining All 3 dataframes

In [3]:
code_slam_nimda_df = pd.concat([code_redi_df,slammer_df,nimda_df])

In [4]:
print(code_slam_nimda_df.shape)
code_slam_nimda_df.head()

(24449, 22)


,H+M,H,M,S,A - no,W - no,A - NLRI,W - NLRI,Mean AS PATH,Max AS PATH,...,W - Dupli,W -Implicit,Avg Edit D,Max Edit D,IAT,IGP,EGP,Incomplete,Packet size,Label
0,0,0,0,8,58,4,130,29,6,13,...,53,8,13,1.0,6,38,0,20,248,-1
1,1,0,1,2,58,7,107,25,6,12,...,83,3,12,1.0,6,48,0,10,232,-1
2,2,0,2,20,43,6,68,22,5,8,...,27,2,8,0.8,6,42,0,1,225,-1
3,3,0,3,0,60,7,119,54,6,10,...,95,21,10,1.1,6,58,0,2,246,-1
4,4,0,4,10,52,6,102,52,5,8,...,52,23,8,0.9,6,48,0,4,241,-1


### Splitting data for training and testing

In [5]:
from sklearn.model_selection import train_test_split

### Normalizing

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
sc = StandardScaler()

In [8]:
X = sc.fit_transform(code_slam_nimda_df.iloc[:,4:21])
X = pd.DataFrame(data=X, columns = code_slam_nimda_df.columns[4:21])
y = code_slam_nimda_df.iloc[:,21]

In [9]:
y.value_counts()

-1    19484
 1     4965
Name: Label, dtype: int64

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.001, random_state=1, stratify=y)
# Stratify so test and train have similar proportion of both classes

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
rf = RandomForestClassifier(n_estimators=100, bootstrap=True, max_features='sqrt' )

In [13]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [14]:
from sklearn.metrics import roc_auc_score

In [15]:
rf_prob = rf.predict_proba(X_test)

In [16]:
rf_prob = rf_prob[:,1]

In [17]:
roc = roc_auc_score(y_test, rf_prob)

In [18]:
roc

0.9600000000000001

### Feature importance

In [19]:
features = pd.DataFrame({'feature': list(X_train.columns), 'importance': rf.feature_importances_}).sort_values(by="importance", ascending=False)

In [20]:
list(features.index)

[0, 13, 3, 2, 8, 11, 15, 16, 7, 9, 1, 5, 10, 12, 4, 6, 14]